In [1]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.utils import embedding_functions
import ollama

/opt/anaconda3/envs/nlp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
client = chromadb.Client()
collection = client.create_collection("recipes")

In [3]:
dataset = load_dataset("AkashPS11/recipes_data_food.com", split="train")
embedder = SentenceTransformer("all-MiniLM-L6-v2")

Generating train split: 100%|██████████| 1048543/1048543 [00:01<00:00, 602299.78 examples/s]


In [11]:
for item in dataset[0]:
    print(item)

RecipeId
Barcode
Name
AuthorId
AuthorName
CookTime
PrepTime
TotalTime
DatePublished
Description
Images
RecipeCategory
Keywords
RecipeIngredientQuantities
RecipeIngredientParts
AggregatedRating
ReviewCount
Calories
FatContent
SaturatedFatContent
CholesterolContent
SodiumContent
CarbohydrateContent
FiberContent
SugarContent
ProteinContent
RecipeServings
RecipeYield
RecipeInstructions


In [12]:
texts = [item for item in dataset["RecipeInstructions"][:500]]  # возьми первые 500 для теста
embeddings = embedder.encode(texts)

collection.add(
    documents=texts,
    embeddings=embeddings,
    ids=[f"recipe-{i}" for i in range(len(texts))]
)

In [22]:
def rag_query(query: str, top_k: int = 3, model: str = "gemma2"):
    query_emb = embedder.encode([query])[0]
    results = collection.query(query_embeddings=[query_emb], n_results=top_k)
    context = "\n\n".join(results["documents"][0])
    # print(context)
    prompt = f"""
Ты — помощник, который отвечает на вопросы о рецептах.
Используй приведённые рецепты как контекст:

{context}

Вопрос: {query}
Ответ:
"""
    response = ollama.chat(model=model, messages=[{"role": "user", "content": prompt}])
    return response["message"]["content"]

In [23]:
answer = rag_query("How to cook chocolate cookies?", model="gemma2")
print(answer)

Here's how to make chocolate cookies based on the provided recipe:

1. **Cream butter, sugar, and vanilla:**  Start by combining softened butter, sugar, and vanilla extract in a bowl and mixing them well until they form a smooth paste. 
2. **Add flour gradually:** On low speed, slowly add the flour to the butter mixture, beating it until just combined. Don't overmix!
3. **Mix in chocolate chips:** Stir in half of the chocolate chips into the dough.
4. **Chill the dough:** Cover the bowl with plastic wrap and refrigerate the dough until it's firm but not rock hard (this usually takes about 30 minutes). This helps prevent the cookies from spreading too much while baking.
5. **Shape cookies:** Once chilled, scoop heaping tablespoons of dough and roll them into 3-inch long logs that are about half an inch thick.
6. **Bake:** Place the cookie logs onto lightly greased baking sheets. Bake in a preheated oven at 350 degrees Fahrenheit for about 8 minutes, or until they turn golden brown.
7. *